In [1]:
%cd ..

/misc/vlgscratch4/LakeGroup/wentao/multimodal-baby


In [2]:
import itertools
import functools
from collections import namedtuple
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from multimodal.multimodal_lit import MultiModalLitModel
from multimodal.multimodal_data_module import PAD_TOKEN_ID, UNK_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID
from ngram import NGramModel
from multimodal.utils import map_structure
from analysis_tools.processing import examples_from_dataloader, get_pos_tags, get_model_items, ModelItems, run_model_on_data_batches
from analysis_tools.sumdata import *
from analysis_tools.build_data import build_data
from analysis_tools.pos_tags import *
from analysis_tools.utils import torch_cache


figsize = (13, 12)
sns.set_theme(
    style='white',
    font='monospace',
    rc={
        'figure.figsize': figsize,
        'axes.grid': False,
        'xtick.bottom': False,
        'xtick.top': False,
        'ytick.left': False,
        'ytick.right': False,
        'axes.spines.left': False,
        'axes.spines.bottom': False,
        'axes.spines.right': False,
        'axes.spines.top': False,
    }
)

np.set_printoptions(suppress=True, precision=2, linewidth=120)
pd.options.display.width = 120

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pos_mapping = pos_mappings[2][1]

used_poses = ["noun", "verb"]

/usr/local/stow/python-3.9/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [3]:
def pos_tagged_seq_repr(tokens, pos_tags):
    return ' '.join(f'{token}/{pos}' for token, pos in zip(tokens, pos_tags))


class Template(namedtuple('TemplateTuple', ['seq', 'pos', 'idx'])):
    """A template where the token at idx of seq is to be filled.
    """

    def __str__(self):
        return ' '.join(
            '_' * len(token) if i == self.idx else token
            for i, (token_id, token) in enumerate(
                (token_id, idx2word[token_id]) for token_id in self.seq)
            if token_id not in [SOS_TOKEN_ID, EOS_TOKEN_ID])


def templates_from_example(token_ids, pos_tags):
    for i, (token_id, pos) in enumerate(zip(token_ids, pos_tags)):
        if pos_mapping[pos] in used_poses:
            yield Template(token_ids, pos_tags, i)


def templates_from_examples(examples, pos_tags, print_pos_tagged_seq=False):
    tot, cnt_filtered = 0, 0

    for example, y_pos_tags in zip(examples, pos_tags):
        x, y, y_len, raw_y = example
        tot += 1
        if print_pos_tagged_seq:
            print(pos_tagged_seq_repr(raw_y[0].split(), y_pos_tags[1:]))
        cnt_present_tokens = sum((pos_mapping[pos] not in ["."] for pos in y_pos_tags))
        if cnt_present_tokens <= 2:
            continue
        y_len = y_len.item()
        y = y[:y_len].tolist()
        if UNK_TOKEN_ID in y:
            continue

        cnt_filtered += 1
        yield example, templates_from_example(y, y_pos_tags)

    print(f'filtered {cnt_filtered} / {tot} = {cnt_filtered / tot :.2%} examples')


def joined_templates_from_examples(*args, **kwargs):
    for example, example_templates in templates_from_examples(*args, **kwargs):
        for template in example_templates:
            yield example, template


def construct_batch_by_filling_template(x, template, word_ids):
    batch_size = len(word_ids)
    x_batch = x.unsqueeze(0).expand(batch_size, *([-1] * x.dim()))
    y = torch.tensor(template.seq)
    y_batch = y.unsqueeze(0).repeat(batch_size, *([1] * y.dim()))
    y_batch[:, template.idx] = torch.tensor(word_ids)
    y_len = torch.tensor(len(template.seq))
    y_len_batch = y_len.unsqueeze(0).expand(batch_size, *([-1] * y_len.dim()))
    raw_y_batch = [[]] * batch_size
    return x_batch, y_batch, y_len_batch, raw_y_batch


def run_model_on_template(model, example, template, word_ids, batch_size=256):
    """Run model and get the whole sentence losses on template filled with every word_id in word_ids
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        batch_size: the batch size
    Returns:
        losses: an np.ndarray of length len(word_ids)
    """

    x, y, y_len, raw_y = example
    x = x.to(device=device)

    batches = (construct_batch_by_filling_template(x, template, word_ids[i : i + batch_size])
               for i in range(0, len(word_ids), batch_size))
    losses = []
    for batch in run_model_on_data_batches(model, batches, use_tqdm=False):
        outputs, loss = batch[4:]
        loss = loss.sum(-1)
        losses.append(loss.detach())
    losses = torch.cat(losses, 0)
    return losses


def get_prob_dist(model, example, template, word_ids, word_cats, batch_size=256):
    """Run model on template and get the probability distribution of categories;
    check whether model gives higher probability to the correct category.
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        word_cats: np.ndarray of same length as word_ids; the categories of each word in word_ids
    Returns:
        pd.Series with categories indexing the probability distribution
    """

    losses = run_model_on_template(model, example, template, word_ids, batch_size=batch_size)
    probs = F.softmax(losses, 0)
    probs_ser = pd.Series(probs.cpu())
    return probs_ser.groupby(word_cats).sum()

In [4]:
# build data and vocab according to the model
data, args = build_data(return_args=True)
dataset_name = args.dataset
vocab = data.datasets['train'].vocab
vocab_size = len(vocab)
print(f'vocab_size = {vocab_size}')
word2idx = vocab
idx2word = [None] * vocab_size
for word, idx in word2idx.items():
    idx2word[idx] = word

my_batch_size = 256
dataloader_fns = {
    'train': lambda: data.train_dataloader(batch_size=my_batch_size, shuffle=False, drop_last=False),
    'val': lambda: data.val_dataloader(batch_size=my_batch_size)[0],
    'test': lambda: data.test_dataloader(batch_size=my_batch_size)[0],
}

checkpoint_path = "checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt"
print(f"load model from {checkpoint_path}")
model = MultiModalLitModel.load_from_checkpoint(checkpoint_path, map_location=device)
model.to(device)

# get the pos tags of all words in vocab
# by the index of model_items produced by get_model_items
train_dataloader_fn = dataloader_fns['train']
train_pos_tags = get_pos_tags(train_dataloader_fn(), dataset_name, 'train')
train_model_items = torch_cache(checkpoint_path + f'.train.cache.pt')(get_model_items)(
    model, train_dataloader_fn(), train_pos_tags, ignore_all_token_items=True)
idx2pos = np.full(vocab_size, '.', dtype='O')
for token_id, pos in train_model_items.token_items.index:
    idx2pos[token_id] = pos
idx2pos2 = np.vectorize(pos_mapping.get)(idx2pos)
for pos in sorted(set(pos_mapping.values())):
    pos_word_ids = np.nonzero(idx2pos2 == pos)[0]
    print(f'#{pos:15s}: {len(pos_word_ids)}')
word_ids = np.nonzero(np.isin(idx2pos2, used_poses))[0]
word_cats = idx2pos2[word_ids]

/home/ww2135/.local/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


Using base transforms
Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created. Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Vocabulary file already exists. Skipping this step.
Calling setup!
vocab_size = 2350
load model from checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt
load cached pos tags: dataset_cache/saycam/t

In [5]:
splits = ['val']

for split in splits:
    dataloader_fn = dataloader_fns[split]
    examples = examples_from_dataloader(dataloader_fn())
    pos_tags = get_pos_tags(dataloader_fn(), dataset_name, split)
    golds, preds, corrects = [], [], []
    for example, template in joined_templates_from_examples(examples, pos_tags):
        print(template)
        gold = pos_mapping[template.pos[template.idx]]
        prob_dist = get_prob_dist(model, example, template, word_ids, word_cats)
        print(prob_dist)
        argmax_idx = prob_dist.argmax()
        pred = prob_dist.index[argmax_idx]
        correct = pred == gold
        print(f'{gold = }, {pred = }, {correct = }')
        golds.append(gold)
        preds.append(pred)
        corrects.append(correct)
    print(f'label distribution: ')
    for pos in used_poses:
        cnt = sum(gold == pos for gold in golds)
        print(f'{pos}: {cnt} / {len(golds)} = {cnt / len(golds):.2%}')
    print(f'accuracy: {sum(corrects)} / {len(corrects)} = {sum(corrects) / len(corrects):.2%}')

load cached pos tags: dataset_cache/saycam/val.pos.cache
we should ____ on some lights , huh ?
noun    0.846758
verb    0.153242
dtype: float32
gold = 'verb', pred = 'noun', correct = False
we should turn on some ______ , huh ?
noun    0.758857
verb    0.241143
dtype: float32
gold = 'noun', pred = 'noun', correct = True
here , ___ me show you .
noun    0.993831
verb    0.006169
dtype: float32
gold = 'verb', pred = 'noun', correct = False
here , let me ____ you .
noun    0.98374
verb    0.01626
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yeah it __ this this one that 's really
noun    0.775444
verb    0.224556
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yeah it 's this this ___ that 's really
noun    0.954641
verb    0.045360
dtype: float32
gold = 'noun', pred = 'noun', correct = True
yeah it 's this this one that __ really
noun    0.902222
verb    0.097778
dtype: float32
gold = 'verb', pred = 'noun', correct = False
what __ wrong with it ? "
noun    0.

noun    0.887726
verb    0.112274
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yea , ___ you gon na walk somewhere ?
noun    0.739592
verb    0.260408
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yea , are you ___ na walk somewhere ?
noun    0.994849
verb    0.005151
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yea , are you gon na ____ somewhere ?
noun    0.903914
verb    0.096086
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yes , it __ getting there .
noun    0.893539
verb    0.106461
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yes , it 's _______ there .
noun    0.957081
verb    0.042919
dtype: float32
gold = 'verb', pred = 'noun', correct = False
that __ right .
noun    0.850569
verb    0.149430
dtype: float32
gold = 'verb', pred = 'noun', correct = False
that ___ a bit much .
noun    0.898318
verb    0.101682
dtype: float32
gold = 'verb', pred = 'noun', correct = False
that was a ___ much .
noun    0.

noun    0.984723
verb    0.015277
dtype: float32
gold = 'noun', pred = 'noun', correct = True
yeah if it 's burnt toast it __ all black and it does n't taste good .
noun    0.922577
verb    0.077423
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yeah if it 's burnt toast it 's all black and it ____ n't taste good .
noun    0.999635
verb    0.000365
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yeah if it 's burnt toast it 's all black and it does n't _____ good .
noun    0.930925
verb    0.069075
dtype: float32
gold = 'verb', pred = 'noun', correct = False
and here __ the flowers .
noun    0.975913
verb    0.024087
dtype: float32
gold = 'verb', pred = 'noun', correct = False
and here 's the _______ .
noun    0.373517
verb    0.626483
dtype: float32
gold = 'noun', pred = 'verb', correct = False
there it __ , yeah .
noun    0.949356
verb    0.050644
dtype: float32
gold = 'verb', pred = 'noun', correct = False
__ you want
noun    0.639799
verb    0.360201
dtyp

noun    0.990461
verb    0.009539
dtype: float32
gold = 'verb', pred = 'noun', correct = False
now , we 're going to put on your _____ and shoes .
noun    0.870553
verb    0.129447
dtype: float32
gold = 'noun', pred = 'noun', correct = True
now , we 're going to put on your socks and _____ .
noun    0.590638
verb    0.409362
dtype: float32
gold = 'noun', pred = 'noun', correct = True
that __ right .
noun    0.777031
verb    0.222969
dtype: float32
gold = 'verb', pred = 'noun', correct = False
you wanna ___ the boys and the , alright , alright we will do the boy and that ’s enough for drums on the
noun    0.99221
verb    0.00779
dtype: float32
gold = 'verb', pred = 'noun', correct = False
you wanna see the ____ and the , alright , alright we will do the boy and that ’s enough for drums on the
noun    0.511867
verb    0.488133
dtype: float32
gold = 'noun', pred = 'noun', correct = True
you wanna see the boys and the , alright , alright we will __ the boy and that ’s enough for drums on t

noun    0.999541
verb    0.000459
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yeah , unless you are _____ to draw some more .
noun    0.997316
verb    0.002684
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yeah , unless you are going to ____ some more .
noun    0.998509
verb    0.001491
dtype: float32
gold = 'verb', pred = 'noun', correct = False
you ____ lots of coins ?
noun    0.908273
verb    0.091727
dtype: float32
gold = 'verb', pred = 'noun', correct = False
you have ____ of coins ?
noun    0.962282
verb    0.037718
dtype: float32
gold = 'noun', pred = 'noun', correct = True
you have lots of _____ ?
noun    0.60969
verb    0.39031
dtype: float32
gold = 'noun', pred = 'noun', correct = True
yeah , you ____ a cold , do n't you ?
noun    0.887491
verb    0.112509
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yeah , you have a cold , __ n't you ?
noun    0.86914
verb    0.13086
dtype: float32
gold = 'verb', pred = 'noun', correct = False

...

noun    0.999421
verb    0.000579
dtype: float32
gold = 'verb', pred = 'noun', correct = False
we have two more colors to make , and what ______ are those ?
noun    0.439035
verb    0.560965
dtype: float32
gold = 'noun', pred = 'verb', correct = False
we have two more colors to make , and what colors ___ those ?
noun    0.995034
verb    0.004965
dtype: float32
gold = 'verb', pred = 'noun', correct = False
no you ____ to .
noun    0.985241
verb    0.014759
dtype: float32
gold = 'verb', pred = 'noun', correct = False
we ___ getting there .
noun    0.848766
verb    0.151234
dtype: float32
gold = 'verb', pred = 'noun', correct = False
we are _______ there .
noun    0.890713
verb    0.109287
dtype: float32
gold = 'verb', pred = 'noun', correct = False
why __ n't we go to a color page ?
noun    0.988106
verb    0.011894
dtype: float32
gold = 'verb', pred = 'noun', correct = False
why do n't we __ to a color page ?
noun    0.90655
verb    0.09345
dtype: float32
gold = 'verb', pred = 'noun', c

noun    0.535363
verb    0.464637
dtype: float32
gold = 'noun', pred = 'noun', correct = True
___ that right ?
noun    0.89678
verb    0.10322
dtype: float32
gold = 'verb', pred = 'noun', correct = False
__ you have a friend ?
noun    0.7266
verb    0.2734
dtype: float32
gold = 'verb', pred = 'noun', correct = False
do you ____ a friend ?
noun    0.89913
verb    0.10087
dtype: float32
gold = 'verb', pred = 'noun', correct = False
do you have a ______ ?
noun    0.455131
verb    0.544869
dtype: float32
gold = 'noun', pred = 'verb', correct = False
yeah , we ____ it in the the car , but we can sing it here , too .
noun    0.987876
verb    0.012124
dtype: float32
gold = 'verb', pred = 'noun', correct = False
yeah , we sing it in the the ___ , but we can sing it here , too .
noun    0.579446
verb    0.420554
dtype: float32
gold = 'noun', pred = 'noun', correct = True
yeah , we sing it in the the car , but we can ____ it here , too .
noun    0.878778
verb    0.121222
dtype: float32
gold = 'v

noun    0.993596
verb    0.006404
dtype: float32
gold = 'verb', pred = 'noun', correct = False
if you pour it all out you ____ to put it back in .
noun    0.998685
verb    0.001315
dtype: float32
gold = 'verb', pred = 'noun', correct = False
if you pour it all out you have to ___ it back in .
noun    0.979647
verb    0.020353
dtype: float32
gold = 'verb', pred = 'noun', correct = False
hm ok you ____ me to do georgia ?
noun    0.957157
verb    0.042843
dtype: float32
gold = 'verb', pred = 'noun', correct = False
hm ok you want me to __ georgia ?
noun    0.834336
verb    0.165664
dtype: float32
gold = 'verb', pred = 'noun', correct = False
hm ok you want me to do _______ ?
noun    0.574699
verb    0.425301
dtype: float32
gold = 'noun', pred = 'noun', correct = True
___ out of order , but -- and it 's a different because we 're not going to the store .
noun    0.813073
verb    0.186927
dtype: float32
gold = 'noun', pred = 'noun', correct = True
bit out of _____ , but -- and it 's a diffe

noun    0.862819
verb    0.137181
dtype: float32
gold = 'verb', pred = 'noun', correct = False
does it swim in the ___ ?
noun    0.620542
verb    0.379458
dtype: float32
gold = 'noun', pred = 'noun', correct = True
the _____ sound .
noun    0.656112
verb    0.343888
dtype: float32
gold = 'noun', pred = 'noun', correct = True
the kitty _____ .
noun    0.857747
verb    0.142253
dtype: float32
gold = 'noun', pred = 'noun', correct = True
____ , see , let 's help .
noun    0.552038
verb    0.447962
dtype: float32
gold = 'verb', pred = 'noun', correct = False
look , ___ , let 's help .
noun    0.441242
verb    0.558758
dtype: float32
gold = 'verb', pred = 'verb', correct = True
look , see , ___ 's help .
noun    0.813286
verb    0.186714
dtype: float32
gold = 'verb', pred = 'noun', correct = False
look , see , let 's ____ .
noun    0.744996
verb    0.255004
dtype: float32
gold = 'verb', pred = 'noun', correct = False
you wanna ___ it ?
noun    0.977476
verb    0.022524
dtype: float32
gold =

noun    0.779975
verb    0.220025
dtype: float32
gold = 'verb', pred = 'noun', correct = False
____ some more ?
noun    0.883443
verb    0.116557
dtype: float32
gold = 'verb', pred = 'noun', correct = False
we can ____ your belly but not your shirt .
noun    0.950579
verb    0.049421
dtype: float32
gold = 'verb', pred = 'noun', correct = False
we can wash your _____ but not your shirt .
noun    0.611248
verb    0.388752
dtype: float32
gold = 'noun', pred = 'noun', correct = True
we can wash your belly but not your _____ .
noun    0.981747
verb    0.018253
dtype: float32
gold = 'noun', pred = 'noun', correct = True
filtered 1087 / 1874 = 58.00% examples
label distribution: 
noun: 987 / 2867 = 34.43%
verb: 1880 / 2867 = 65.57%
accuracy: 833 / 2867 = 29.05%
